In [ ]:
from PyQt5.QtWidgets import QOpenGLWidget
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import Qt,QPoint,QLineF
from OpenGL.GL import *
from OpenGL.arrays import vbo
from OpenGL.GLU import *
from OpenGL.GLUT import *
from numpy import *
import math
import form
from form import Ui_Form
from Camera import camera
from Vector3f import *
from Light import Light
from Optimizermaster.FileHandler import FileHandler
import sys
from ctypes import sizeof, c_float, c_void_p, c_uint
file=None#导入文件
vertex_data=[]
float_size = sizeof(c_float)
record_len       =         0 * float_size
vertex_offset    = c_void_p(0 * float_size)
class GLWidget(QOpenGLWidget):
    def __init__(self, *args, **kwargs):
        super(GLWidget, self).__init__(*args, **kwargs)
        self.setGeometry(QtCore.QRect(10, 10,700, 700))#窗口位置和大小
        self.isMove=False
        self.isRotate=False
    def initializeGL(self):
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT);
        self.camera=camera()
        self.light=Light()
    def paintGL(self):
        self.camera.Update()
        self.light.Update()
        #glutWireTeapot(0.5)
        if file==None:
            print('mesh is none')
        else:
            #self.testdraw()
            self.VBO()
    def testdraw(self):
        glBegin(GL_TRIANGLES)
        i=0
        for mesh in file[0]['mesh']:
            if i%3==0:
                glNormal3f(file[0]['normal'][i][0],file[0]['normal'][i][1],file[0]['normal'][i][2])
            glVertex3f(mesh[0],mesh[1],mesh[2])
            i+=1
            '''glVertex3f(mesh[1][0],mesh[1][1],mesh[1][2])
            glVertex3f(mesh[2][0],mesh[2][1],mesh[2][2])'''
        glEnd()
    def VBO(self):
        
        vertexBuffer=[0]
        data_buffer = (c_float*len(vertex_data))(*vertex_data)
        glGenBuffers(1,vertexBuffer)
        glBindBuffer(GL_ARRAY_BUFFER,vertexBuffer[0] )
        
        glBufferData(GL_ARRAY_BUFFER,data_buffer,id(vertex_data),GL_STATIC_DRAW)
        
        glEnableClientState(GL_VERTEX_ARRAY)
        glBindBuffer(GL_ARRAY_BUFFER, vertexBuffer[0])
        glVertexPointer(3, GL_FLOAT, record_len, vertex_offset)
        glDrawArrays(GL_TRIANGLES,0,len(vertex_data))
        glDisableClientState(GL_VERTEX_ARRAY)
        
        glDeleteBuffers(1,vertexBuffer)
        
    def mousePressEvent(self,event):
        self.myMousePosition=event.pos()
        if event.button()==Qt.LeftButton:
            self.isMove=True
        if event.button()==Qt.RightButton:
            self.isRotate=True
        #self.update()
    def wheelEvent(self,event):
        if event.angleDelta().y()>0:
            print('add')
        if event.angleDelta().y()<0:
            print('sub')
    def mouseReleaseEvent(self,event):
        if event.button()==Qt.LeftButton:
            self.isMove=False
        if event.button()==Qt.RightButton:
            self.isRotate=False
        #print('mPos:(X:'+str(self.camera.mPos.X)+',Y:'+str(self.camera.mPos.Y)+',Z:'+str(self.camera.mPos.Z)+')')
        #print('mViewCenter:(X:'+str(self.camera.mViewCenter.X)+',Y:'+str(self.camera.mViewCenter.Y)+',Z:'+str(self.camera.mViewCenter.Z)+')')
    def mouseMoveEvent(self,event):
        if self.isMove:
            moveX=event.pos().x()-self.myMousePosition.x()
            moveY=event.pos().y()-self.myMousePosition.y()
            self.camera.Move(moveX,moveY)
            self.myMousePosition=event.pos()
        if self.isRotate:
            moveX=event.pos().x()-self.myMousePosition.x()
            moveY=event.pos().y()-self.myMousePosition.y()
            self.camera.Yaw(-moveX)
            self.camera.Pitch(-moveY)
            self.myMousePosition=event.pos()
        self.update()

class myform(QtWidgets.QWidget,Ui_Form):
    
    def __init__(self):
        super(myform,self).__init__()
        self.setupUi(self)
        self.graphicsView1=GLWidget(self)
        self.btn=QtWidgets.QPushButton(self)
        self.btn.setGeometry(QtCore.QRect(710, 40, 75, 23))
        self.btn.setObjectName("pushButton")
        self.btn.setText("打开文件")
        self.btn.clicked.connect(self.getfile)
    def getfile(self):
        global file
        dlg=QtWidgets.QFileDialog()
        f=dlg.getOpenFileName(self,'Open File','.\\Optimizermaster','STL File (*.stl)')
        file=FileHandler().load_mesh(f[0])
        for mesh in file[0]['mesh']:
            vertex_data.append(mesh[0])
            vertex_data.append(mesh[1])
            vertex_data.append(mesh[2])
app=QtWidgets.QApplication(sys.argv)
mywindow=myform()
mywindow.show()
app.exec_()
sys.exit()

mesh is none
mesh is none
mesh is none
b'>7\x00\x00'
(14142,)


TypeError: Don't know how to convert None to an array type

TypeError: Don't know how to convert None to an array type

TypeError: Don't know how to convert None to an array type

TypeError: Don't know how to convert None to an array type

File type is not supported.


In [ ]:
dir(QtWidgets.QWidget)

In [ ]:
help(QOpenGLWidget.resizeGL)

In [ ]:
print(cos(60*math.pi/180))

In [ ]:
i=10.0
print(sys.getsizeof(i))
print(type(i))

In [ ]:
inn=[]
dir(inn)